In [55]:
%run runaway_functionsv3
%matplotlib qt
# cl = Cluster('LP_1487')
# # cld = ClusterDias('Berkeley_97')
# # plot = cl.plot_cmd(isochrones=[Isochrone(cld)])
# plot = cl.plot_traceback_psr()
# show_annotation_manager(plot)


In [8]:
dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
dias2021['logage'].min(), dias2021['logage'].max()

(6.668, 9.999)

In [4]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('Alessi_Teutsch_5')
# print(latex_text(cl))
plot_cmd = cl.plot_cmd()

In [5]:
show_annotation_manager(plot_cmd)

In [6]:
cl.runaways()

Name,RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,e_v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,K,K,km / s,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
Gaia DR3 2203047987818327936,331.44191766530,60.41928497013,828.91674800,11066.8,11060.0,61.90181727275718,2.4882423261991082,63.26,25.67,--,--,2203047987818327936,1.1853,0.0174,-14.690,-12.713,0.023,0.019,1.068,12.138815,0.475061,12.314358,11.839296,816.44787600,840.95831300,0.002764,0.002868,0.003791,0.006658,"331.4419176653,60.41928497013,828.916748",-12.638,0.402,-9.407,0.398,4.3845,-36.54,29.69,-13.05,40.83,262676.24215,53.52,1.621e+05,98.77,1.457e+05,39.61,--,--


In [19]:
stars_in_region  = cl.stars_in_region()
# stars_in_region  = cl.runaways()
from uncertainties import ufloat, unumpy
rgeo = stars_in_region['rgeo'].value
b_rgeo = stars_in_region['b_rgeo'].value
B_rgeo = stars_in_region['B_rgeo'].value
rmRA = unumpy.uarray(stars_in_region['rmRA'].value,stars_in_region['e_rmRA'].value)
rmDE = unumpy.uarray(stars_in_region['rmDE'].value,stars_in_region['e_rmDE'].value)
def pm(pmRA,pmDE):
    µ = (pmRA**2+pmDE**2)**0.5
    return µ

def v(µ,dist):
    #dist should be in pc
    v = (µ*4.74*dist/1000)
    return v
rm = pm(rmRA,rmDE)
v_trans  = unumpy.nominal_values(v(rm,rgeo))*u.km/u.s
v_trans_upper = unumpy.nominal_values(v(rm,B_rgeo))*u.km/u.s
v_trans_lower = unumpy.nominal_values(v(rm,b_rgeo))*u.km/u.s

e_v_trans = abs((v_trans_lower-v_trans_upper)/2)


In [9]:
cl.runaways()

Name,RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,e_v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,K,K,km / s,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
Gaia DR3 2008384504995669248,339.89581711423,58.99547917199,2620.36230000,9168.0,10980.0,29.98008203419036,1.7051992469891282,--,--,--,--,2008384504995669248,0.3560,0.0241,-0.379,-1.352,0.025,0.024,1.086,15.384143,1.182070,15.883185,14.701116,2461.53345000,2759.61401000,0.00277,0.003852,0.004229,0.008081,"339.89581711423,58.99547917199,2620.3623",2.354,0.124,0.532,0.123,3.7061,--,--,--,--,13221.76503,3.482,6055,14.81,1.044e+04,18.25,--,--


In [1]:
from uncertainties import ufloat
pmRA = ufloat(2.354	,0.146)
pmDE = ufloat(0.532,0.175)
rgeo = 2620
b_rgeo = 2620-139
B_rego = 2620+159
# 2.354	0.124	0.532	0.123
# pmRA = ufloat(2.374, 0.146)
# pmDE = ufloat(0.563, 0.175)
# rgeo = 2620
# b_rgeo = 2620-139
# B_rego = 2620+159

def pm(pmRA,pmDE):
    µ = (pmRA**2+pmDE**2)**0.5
    return µ

def v(µ,dist):
    #dist should be in pc
    v = (µ*4.74*dist/1000)
    return v

# dµ = (dµa*µa/µ+dµd*µd/µ)

# v(µ,828.91674800), v(dµ, 828)

v_trans = (v(pm(pmRA,pmDE),rgeo).nominal_value)
v_trans_upper = v(pm(pmRA,pmDE),B_rego).nominal_value
v_trans_lower = v(pm(pmRA,pmDE),b_rgeo).nominal_value
v_trans, abs(v_trans_lower-v_trans),abs(v_trans_upper-v_trans), (v_trans_upper-v_trans_lower)/2

(29.971121387701356, 1.5900709438513339, 1.818858130016988, 1.704464536934161)

In [30]:
rgeo = 2620+159
pmRA = 2.374 + 0.146
pmDE = 0.563 + 0.175

v(pm(pmRA,pmDE),rgeo)


34.58879317071774

In [28]:
dv = (30.1)*4.74*np.sqrt(((0.146/2.374)**2+(0.175/0.563)**2)
                         +(.159/2.620)**2)
dv

46.02942613847946

In [18]:
v(pm(pmRA,pmDE),rgeo)

29.971121387701356+/-1.8322825288273021

In [36]:
%run runaway_functionsv3
def generate_kinematics_latex(cl):
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # Cluster part
    cluster_part = Table(cl.all)

    # Create the combined columns
    name_column = [name.replace('_', ' ') for name in cluster_part['Cluster']]
    ra_column   = [round(ra,3) for ra in cluster_part['RA_ICRS']]
    dec_column   = [round(dec,3) for dec in cluster_part['DE_ICRS']]
    distance_column = [f"${dist:.0f}\pm{e_dist:.0f}$" for dist, e_dist in zip(cluster_part['Dist'], cluster_part['e_Dist'])]
    pmRA_column = [f"${pmRA:.1f}\pm{e_pmRA:.1f}$" for pmRA, e_pmRA in zip(cluster_part['pmRA'], cluster_part['e_pmRA'])]
    pmDE_column = [f"${pmDE:.1f}\pm{e_pmDE:.1f}$" for pmDE, e_pmDE in zip(cluster_part['pmDE'], cluster_part['e_pmDE'])]
    vr_column = [f"${RV:.1f}\pm{e_RV:.1f}$" for RV, e_RV in zip(cluster_part['RV'], cluster_part['e_RV'])]


    # Add the new columns to the table
    cluster_part['Name'] = name_column
    cluster_part['RA_ICRS'] = ra_column
    cluster_part['DE_ICRS'] = dec_column
    cluster_part['d'] = Column(distance_column, unit='(pc)')
    cluster_part[r'$\mu_{\alpha}^*$'] = Column(pmRA_column, unit=u.mas/u.yr)
    cluster_part[r'$\mu_{\delta}$'] = Column(pmDE_column, unit=u.mas/u.yr)
    cluster_part[r'$v_r$'] = Column(vr_column, unit=u.km/u.s)

    cluster_part.rename_column('RA_ICRS',r'$\alpha$')
    cluster_part.rename_column('DE_ICRS',r'$\delta$')
    cluster_part  = cluster_part['Name',r'$\alpha$',r'$\delta$','d',r'$\mu_{\alpha}^*$',r'$\mu_{\delta}$',r'$v_r$']


    # Runaway part
    runaway_part = cl.runaways()
    # Create the combined columns
    name_column = [str(name) for name in runaway_part['Source']]
    ra_column   = [round(ra,3) for ra in runaway_part['RA_ICRS_1']]
    dec_column   = [round(dec,3) for dec in runaway_part['DE_ICRS_1']]
    distance_column = [f"${dist:.0f}\pm{e_dist:.0f}$" for dist, e_dist in zip(runaway_part['rgeo'], (runaway_part['B_rgeo']-runaway_part['b_rgeo']))]
    pmRA_column = [f"${pmRA:.1f}\pm{e_pmRA:.1f}$" for pmRA, e_pmRA in zip(runaway_part['pmRA'], runaway_part['e_pmRA'])]
    pmDE_column = [f"${pmDE:.1f}\pm{e_pmDE:.1f}$" for pmDE, e_pmDE in zip(runaway_part['pmDE'], runaway_part['e_pmDE'])]
    vr_column = [f"${RV:.1f}\pm{e_RV:.1f}$" for RV, e_RV in zip(runaway_part['RV'], runaway_part['e_RV'])]
    vpec2d_column = [f"${v_pec:.1f}\pm{e_v_pec:.1f}$" for v_pec, e_v_pec in zip(runaway_part['v_pec'], runaway_part['e_v_pec'])]
    # Add the new columns to the table
    runaway_part['Name'] = name_column
    runaway_part['RA_ICRS_1'] = Column(ra_column, unit='deg')
    runaway_part['DE_ICRS_1'] = Column(dec_column, unit='deg')
    runaway_part['d'] = Column(distance_column, unit='pc')
    runaway_part[r'$\mu_{\alpha}^*$'] = Column(pmRA_column, unit=u.mas/u.yr)
    runaway_part[r'$\mu_{\delta}$'] = Column(pmDE_column, unit=u.mas/u.yr)
    runaway_part[r'$v_r$'] = Column(vr_column, unit=u.km/u.s)
    runaway_part[r'$v_\text{trans}$'] = Column(vpec2d_column, unit=u.km/u.s)

    runaway_part.rename_column('RA_ICRS_1',r'$\alpha$')
    runaway_part.rename_column('DE_ICRS_1',r'$\delta$')
    runaway_part  = runaway_part['Name',r'$\alpha$',r'$\delta$','d',r'$\mu_{\alpha}^*$',r'$\mu_{\delta}$',r'$v_r$',r'$v_\text{trans}$']
    
    
    warnings.filterwarnings("ignore", category=MergeConflictWarning)
    kinematic_table=vstack([cluster_part,runaway_part])


    output = io.StringIO()
    
    latexdict_kinematics = {
    'tabletype': 'table*',
    'tablealign': 'h',
    'header_start': r'\hline',
    'data_start': r'\hline',
    'data_end': r'\hline', 
    'caption': f'Kinematics of {cl.name.replace("_"," ")} compared to its runaway'
                +r'\label{tab:' + f'{cl.name}_kinematics' + '}',
    'preamble':r'\fontsize{9pt}{11pt}\selectfont',
                
    }
    
    ascii.write(kinematic_table, output, format='latex', latexdict=latexdict_kinematics)
    kinematics_table_text = output.getvalue()
    output.close()
    kinematics_table_text.replace(r'$--\pm--$','n/a')
    return kinematics_table_text, kinematic_table


def generate_members_latex(cluster, n_members=10):
    # Members table generation
    table = cluster.mymembers
    table.sort('Gmag')
    table = table[:n_members]
    
    formatted_distances = []
    # formatted_parallaxes = []
    formatted_pmras = []
    formatted_pmdes = []
    formatted_gmags = []
    formatted_bprps = []
    
    for row in table:
        distance = row['rgeo']
        upper_error = row['B_rgeo'] - row['rgeo']
        lower_error = -row['b_rgeo'] + row['rgeo']
        # plx = row['Plx']
        # e_plx = row['e_Plx']
        pmra = row['pmRA']
        e_pmra = row['e_pmRA']
        pmde = row['e_pmDE']
        e_pmde = row['e_pmDE']
        gmag = row['Gmag']
        e_gmag = row['e_Gmag']
        bprp = row['BP-RP']
        e_bprp = row['e_BP-RP']
        
        formatted_distance = f"${distance:.0f}^{{+{upper_error:.0f}}}_{{-{lower_error:.0f}}}$"
        formatted_pmra = f"${pmra:.2f}\pm{e_pmra:.2f}$"
        formatted_pmde = f"${pmde:.2f}\pm{e_pmde:.2f}$"
        # formatted_parallax = f"${plx:.4f}\pm{e_plx:.4f}$"
        formatted_gmag = f"${gmag:.3f}\pm{e_gmag:.3f}$"
        formatted_bprp = f"${bprp:.3f}\pm{e_bprp:.3f}$"
        
        formatted_distances.append(formatted_distance)
        # formatted_parallaxes.append(formatted_parallax)
        formatted_pmras.append(formatted_pmra)
        formatted_pmdes.append(formatted_pmde)
        formatted_gmags.append(formatted_gmag)
        formatted_bprps.append(formatted_bprp)

    table['formatted_distance'] = formatted_distances
    # table['formatted_parallax'] = formatted_parallaxes
    table['formatted_pmra'] = formatted_pmras
    table['formatted_pmde'] = formatted_pmdes
    table['formatted_gmag'] = formatted_gmags
    table['formatted_bprp'] = formatted_bprps
    
    t_dict_members = {
        'Gaia DR3 Source': table['Source'],
        r"$r_{\text{geo}}$": table['formatted_distance'],
        # r"$\pi$ (mas)": table['formatted_parallax'],
        r"$G$": table['formatted_gmag'],
        r'$\mu_{\alpha}^*$': table['formatted_pmra'],
        r'$\mu_{\delta}$': table['formatted_pmde'],
        r"$G_{\text{BP}}-G_{\text{RP}}$": table['formatted_bprp']
    }
    
    latexdict_members = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'units':{
                # 'Gaia DR3 Source': table['Source'],
                r"$r_{\text{geo}}$": 'pc',
                # r"$\pi$ (mas)": table['formatted_parallax'],
                r"$G$": 'mag',
                r'$\mu_{\alpha}^*$': 'mas/yr',
                r'$\mu_{\delta}$': 'mas/yr',
                r"$G_{\text{BP}}-G_{\text{RP}}$": 'mag'
            },
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Selected members of {cluster.name.replace("_"," ")}'
                    +r'\label{tab:' + f'{cluster.name}_members' + '}',
        'preamble': r'\fontsize{11pt}{11pt}\selectfont',
    }
    
    astropy_table_members = Table(t_dict_members)

    output = io.StringIO()
    
    ascii.write(astropy_table_members, output, format='latex', latexdict=latexdict_members)
    members_table_text = output.getvalue()
    output.close()
    return members_table_text


def latex_text(cluster, n_members=10):
    kinematics_table_text, kinematics_table = generate_kinematics_latex(cluster)
    run = cluster.runaways()
    cld = ClusterDias(cluster.name)
    ra, dec = cluster.skycoord.ra, cluster.skycoord.dec
    ra_str, dec_str = ra.to_string(format='latex')[1:-1], dec.to_string(format='latex')[1:-1]
    dist_str = str(round(cluster.distance.value,0)) + r"\pm" + str(round(cluster.e_distance.value,0))
    
    intro_text = rf"""\subsection{{Introduction}}
    {cluster.name.replace('_', ' ')} is located at $\alpha = {ra_str}, \delta = {dec_str}$ at a distance of ${dist_str}$ pc.
    {cluster.N} out of the {cld.N} mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{{tab:{cluster.name}_kinematics}}. Some of the brightest memebrs fo the cluster are shown in table \ref{{tab:{cluster.name}_members}}.
    
    \subsection{{CMD and proper motion}}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{{fig:{cluster.name}_cmd_pm}} in the left.
    The runaway star is highlighted with a temperature estimate of {run['Temp. Est'][0]:.0f} using this isochrone. This corresponds to a spectral class of ... .
    The right plot in the figure shows the proper motion diagram showing that the runaway moves with a proper motion that is different from the cluster.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_cmd.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The CMD showing the temperature estimate of the runaway. The isochrone for the temperature estimate is shown in blue \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_cmd_pm}}
    \end{{figure}}    

    \subsection{{Traceback}}
    The runaway star is {run['Name'][0]} whose motion relative to the cluster is shown in figure \ref{{fig:{cluster.name}_traceback.pdf}}. The trajectory of the runaway star relative to the cluster's motion for 100 kyr in past has been depicted, together with the uncertainties in the proper motion. The runaway star is located at a distance of ${run['rgeo'][0]:.0f}^{{{(run['B_rgeo'][0]-run['rgeo'][0]):.0f}}}_{{{(run['b_rgeo'][0]-run['rgeo'][0]):.0f}}}$.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_traceback_clean.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The runaway star ...'s motion with respect to the cluster. The four green lines depict the four extreme cases of the proper motion considering the errors and the green ellipse represents the possible positions of the star 100 kyr ago. \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_traceback_clean&pm}}
    \end{{figure}}
    
    """

    
    
    
    # Combine intro_text, kinematics_table_text, and members_table_text
    full_text = intro_text + "\n\n" + kinematics_table_text + "\n\n" + generate_members_latex(cluster)
    
    
    return full_text



cl = Cluster('Alessi_Teutsch_5')
# cl = Cluster('Berkeley_97')
print(latex_text(cl))

\subsection{Introduction}
    Alessi Teutsch 5 is located at $\alpha = 332^\circ16{}^\prime27.84{}^{\prime\prime}, \delta = 61^\circ05{}^\prime46.32{}^{\prime\prime}$ at a distance of $870.0\pm31.0$ pc.
    59 out of the 147 mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{tab:Alessi_Teutsch_5_kinematics}. Some of the brightest memebrs fo the cluster are shown in table \ref{tab:Alessi_Teutsch_5_members}.
    
    \subsection{CMD and proper motion}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{fig:Alessi_Teutsch_5_cmd_pm} in the left.
    The runaway star is highlighted with a temperature estimate of 11060 using this isochrone. This corresponds to a spectral class of ... .
    The right plot in the figure shows the proper motion diagram showing that the ru

In [20]:
generate_kinematics_latex(cl)[1]

Name,$\alpha$,$\delta$,d,$\mu_{\alpha}^*$,$\mu_{\delta}$,$v_r$,$v_\text{trans}$
,deg,deg,pc,mas / yr,mas / yr,km / s,km / s
str19,float64,float64,str12,str12,str12,str9,str12
Berkeley 97,339.85,59.005,$2769\pm132$,$-2.7\pm0.1$,$-1.9\pm0.1$,$--\pm--$,--
2008384504995669248,339.896,58.995,$2620\pm298$,$-0.4\pm0.0$,$-1.4\pm0.0$,$--\pm--$,$30.0\pm3.2$


In [16]:
cl.runaways()['Temp. Est'][0]

10980.0

In [ ]:
kinem

In [12]:
print(generate_kinematics_latex(cl))

\begin{table*}[h]
\fontsize{9pt}{11pt}\selectfont
\caption{Kinematics of Berkeley 97 compared to its runaway\label{tab:Berkeley_97_kinematics}}
\begin{tabular}{cccccccc}
\hline
Name & $\alpha$ & $\delta$ & d & $\mu_{\alpha}^*$ & $\mu_{\delta}$ & $v_r$ & $v_\text{trans}$ \\
 & $\mathrm{{}^{\circ}}$ & $\mathrm{{}^{\circ}}$ & $\mathrm{pc}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{mas\,yr^{-1}}$ & $\mathrm{km\,s^{-1}}$ & $\mathrm{km\,s^{-1}}$ \\
\hline
Berkeley 97 & 339.85 & 59.005 & $2769\pm132$ & $-2.7\pm0.1$ & $-1.9\pm0.1$ & $--\pm--$ &  \\
2008384504995669248 & 339.896 & 58.995 & $2620\pm298$ & $-0.4\pm0.0$ & $-1.4\pm0.0$ & $--\pm--$ & $30.0\pm3.2$ \\
\hline
\end{tabular}
\end{table*}



In [3]:
cl.plot_pm()
cl.plot_traceback_clean()
cl.plot_traceback_psr()

10pc image downloaded in 4.706837s
50pc image downloaded in 28.883331s


<WCSAxesSubplot: >